# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [15]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [16]:
import os
print(os.getcwd())
cwd=os.getcwd()
xdir=os.path.dirname(cwd)
path=os.path.join(xdir,"output_data", "cities.csv")

C:\Users\jordo\OneDrive\Desktop\git-projects\python-api-challenge\WeatherPy


In [17]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv(path)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,laramie,41.3114,-105.5911,7.53,94,100,11.32,US,1683854163
1,1,hawaiian paradise park,19.5933,-154.9731,25.73,76,100,3.60,US,1683854164
2,2,blackmans bay,-43.0167,147.3167,14.18,74,96,1.16,AU,1683854166
3,3,stanley,54.8680,-1.6985,9.84,97,75,3.09,GB,1683854166
4,4,iqaluit,63.7506,-68.5145,-10.06,92,72,1.67,CA,1683854166


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [18]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points("Lng", "Lat", geo = True, tiles = "OSM", alpha = 0.5, frame_width = 700, 
                                      frame_height = 500, size = "Humidity", color = "City")

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [23]:
# Narrow down cities that fit criteria and drop any results with null values
my_weather = city_data_df.loc[(city_data_df['Max Temp']>12) & (city_data_df['Max Temp']<=26) & 
                              (city_data_df['Wind Speed']<6) & (city_data_df['Humidity']<72)]

# Drop any rows with null values
my_weather = my_weather.dropna()

# Display sample data
my_weather

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,badger,64.8000,-147.5333,16.14,39,75,2.57,US,1683854167
10,10,xinzhi,36.4989,111.7047,20.95,45,8,2.44,CN,1683854168
12,12,dryden,49.7833,-92.7503,18.09,48,99,2.29,CA,1683854168
13,13,apolinario saravia,-24.4328,-63.9954,19.40,59,76,0.76,AR,1683854169
16,16,whitehorse,60.7161,-135.0538,15.01,33,15,5.36,CA,1683853922
...,...,...,...,...,...,...,...,...,...,...
551,551,binzhou,37.3667,118.0167,24.82,33,0,5.54,CN,1683854302
557,557,laojunmiao,39.8333,97.7333,14.45,14,0,3.15,CN,1683854303
558,558,tochio-honcho,37.4667,139.0000,16.97,42,2,1.34,JP,1683854304
567,567,lucheng,31.2336,117.2806,20.41,55,8,1.85,CN,1683854307


### Step 3: Create a new DataFrame called `hotel_df`.

In [24]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = my_weather.copy(deep=True)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
8,8,badger,64.8000,-147.5333,16.14,39,75,2.57,US,1683854167,
10,10,xinzhi,36.4989,111.7047,20.95,45,8,2.44,CN,1683854168,
12,12,dryden,49.7833,-92.7503,18.09,48,99,2.29,CA,1683854168,
13,13,apolinario saravia,-24.4328,-63.9954,19.40,59,76,0.76,AR,1683854169,
16,16,whitehorse,60.7161,-135.0538,15.01,33,15,5.36,CA,1683853922,
...,...,...,...,...,...,...,...,...,...,...,...
551,551,binzhou,37.3667,118.0167,24.82,33,0,5.54,CN,1683854302,
557,557,laojunmiao,39.8333,97.7333,14.45,14,0,3.15,CN,1683854303,
558,558,tochio-honcho,37.4667,139.0000,16.97,42,2,1.34,JP,1683854304,
567,567,lucheng,31.2336,117.2806,20.41,55,8,1.85,CN,1683854307,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [29]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
          "apiKey": geoapify_key,
          "limit": 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
badger - nearest hotel: Hotel North Pole
xinzhi - nearest hotel: No hotel found
dryden - nearest hotel: Holiday Inn Express
apolinario saravia - nearest hotel: No hotel found
whitehorse - nearest hotel: Town & Mountain Hotel
yellowknife - nearest hotel: No hotel found
port macquarie - nearest hotel: The Observatory
thompson - nearest hotel: Thompson Inn
hasaki - nearest hotel: 大新旅館
bethel - nearest hotel: Hampton Inn Danbury
loreto - nearest hotel: No hotel found
new norfolk - nearest hotel: Woodbridge on the Derwent
mount isa - nearest hotel: Ibis Styles
at taj - nearest hotel: No hotel found
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
las lomitas - nearest hotel: Hotel Lomitas
rio grande - nearest hotel: Hotel Vila Moura Executivo
nurota - nearest hotel: Nurota
whangarei - nearest hotel: Grand Hotel
westport - nearest hotel: Norwalk Inn and Conference Center
jidong - nearest hotel: No hotel found
margaret river - nearest hotel: Margaret River Hot

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
8,8,badger,64.8000,-147.5333,16.14,39,75,2.57,US,1683854167,Hotel North Pole
10,10,xinzhi,36.4989,111.7047,20.95,45,8,2.44,CN,1683854168,No hotel found
12,12,dryden,49.7833,-92.7503,18.09,48,99,2.29,CA,1683854168,Holiday Inn Express
13,13,apolinario saravia,-24.4328,-63.9954,19.40,59,76,0.76,AR,1683854169,No hotel found
16,16,whitehorse,60.7161,-135.0538,15.01,33,15,5.36,CA,1683853922,Town & Mountain Hotel
...,...,...,...,...,...,...,...,...,...,...,...
551,551,binzhou,37.3667,118.0167,24.82,33,0,5.54,CN,1683854302,No hotel found
557,557,laojunmiao,39.8333,97.7333,14.45,14,0,3.15,CN,1683854303,No hotel found
558,558,tochio-honcho,37.4667,139.0000,16.97,42,2,1.34,JP,1683854304,ホテル丁字谷
567,567,lucheng,31.2336,117.2806,20.41,55,8,1.85,CN,1683854307,锦怡晓厨


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [33]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", "Lat", geo = True, tiles = "OSM", alpha = 0.5, frame_width = 700, 
                                      frame_height = 500, size = "Humidity", hover_cols = ["Hotel Name", "Country"], color = "City")

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)